In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import pickle
from time import perf_counter 
import seaborn as sn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [3]:
DIR = r'/kaggle/input/dog-vs-cat-fastai/dogscats/dogscats'
IMG_SIZE = 150
batch_size = 60
epochs = 10

## Preparing the Data

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DIR + '/train',
  seed=99,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DIR + '/valid',
  seed=99,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size)

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

#train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(IMG_SIZE, 
                                                              IMG_SIZE,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255)
])

## BUILDING THE MODEL

In [7]:
model = Sequential([
    data_augmentation,
    resize_and_rescale,
    Conv2D(32, (3,3), input_shape = (IMG_SIZE,IMG_SIZE,3), activation = 'relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation = 'relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation = 'relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(2, activation = 'softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [8]:
retVal = model.fit(train_ds, validation_data=valid_ds, epochs=epochs)

In [9]:
model.save('model.h5')